In [58]:
import pickle
import pycaret
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tsfresh import select_features
from pycaret.classification import *
from tsfresh import extract_features
from tsfresh import extract_relevant_features
from sklearn.ensemble import ExtraTreesClassifier
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters
from notebooks_preprocessing_and_dataframe_creation import preprocessing_functions

In [59]:
cmap='seismic'
sns.set_palette(sns.color_palette(cmap))

In [60]:
metric = 'AUC' # F1 or AUC

In [61]:
df = pd.read_pickle('../data/unified_dataframe/data_unprocessed.pkl')
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,label_ttm_stage,label_breq_self_determination,label_sema_negative_feelings,label_ipip_extraversion_category,label_ipip_agreeableness_category,label_ipip_conscientiousness_category,label_ipip_stability_category,label_ipip_intellect_category,label_stai_stress_category,label_panas_negative_affect
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,Maintenance,intrinsic_regulation,NaN,LOW,LOW,HIGH,HIGH,AVERAGE,NaN,NaN
1,621e328667b776a240281372,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Maintenance,intrinsic_regulation,NaN,HIGH,AVERAGE,HIGH,LOW,LOW,NaN,NaN
2,621e326767b776a24012e179,2021-05-24,NaN,46.120,53.968,95.3,14.4,80.0,0.666667,0.925,...,Maintenance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e332267b776a24092a584,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Maintenance,identified_regulation,NaN,LOW,LOW,LOW,AVERAGE,HIGH,NaN,NaN
4,621e333567b776a240a0c217,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Contemplation,intrinsic_regulation,NaN,AVERAGE,HIGH,AVERAGE,LOW,LOW,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7799,621e30b267b776a240c5e13f,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7800,621e312a67b776a240164d59,2022-01-17,34.237581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Contemplation,intrinsic_regulation,NaN,AVERAGE,LOW,HIGH,AVERAGE,LOW,NaN,NaN
7801,621e309b67b776a240b532b0,2022-01-17,31.815133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7802,621e333567b776a240a0c217,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Contemplation,intrinsic_regulation,NaN,AVERAGE,HIGH,AVERAGE,LOW,LOW,NaN,NaN


In [62]:
df_pre = pd.read_pickle('../data/unified_dataframe/data_preprocessed.pkl')
df_pre

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,Steps_hour14,Steps_hour15,Steps_hour16,Steps_hour17,Steps_hour18,Steps_hour19,Steps_hour20,Steps_hour21,Steps_hour22,Steps_hour23
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,95.9,14.8,78.0,0.833333,0.675,...,191.0,33.0,342.0,1712.0,1838.0,160.0,155.0,37.0,31.0,53.0
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,95.9,15.8,80.0,0.833333,0.725,...,120.0,24.0,446.0,377.0,3165.0,332.0,97.0,9.0,24.0,58.0
2,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,95.9,15.8,80.0,0.833333,0.725,...,120.0,24.0,446.0,377.0,3165.0,332.0,97.0,9.0,24.0,58.0
3,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,35.216,95.9,14.6,84.0,0.966667,0.725,...,85.0,477.0,390.0,2821.0,293.0,158.0,66.0,0.0,70.0,74.0
4,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,35.216,95.9,14.8,82.0,0.933333,0.725,...,58.0,296.0,401.0,21.0,3054.0,503.0,411.0,96.0,63.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7799,621e375b67b776a240290cdc,2021-07-22,33.973120,62.388,35.216,95.9,14.6,76.0,0.766667,0.725,...,7.0,60.0,7.0,163.0,92.0,106.0,2034.0,3453.0,49.0,170.0
7800,621e375b67b776a240290cdc,2021-07-23,34.011607,55.542,30.813,95.9,16.6,81.0,0.700000,0.800,...,1013.0,810.0,371.0,833.0,1411.0,93.0,424.0,4029.0,1083.0,42.0
7801,621e375b67b776a240290cdc,2021-07-24,33.687826,69.579,19.407,95.9,17.0,84.0,0.833333,0.900,...,1656.0,1276.0,1903.0,534.0,491.0,583.0,84.0,1103.0,621.0,737.0
7802,621e375b67b776a240290cdc,2021-07-25,34.112386,65.899,22.892,95.9,18.0,84.0,0.833333,0.900,...,42.0,1235.0,716.0,1130.0,1163.0,888.0,245.0,1401.0,54.0,1627.0


# Predict personality extraversion

In [63]:
extraversion = df.drop(columns=['date','startTime', 'endTime', 'label_sema_negative_feelings', 'label_ipip_agreeableness_category', 'label_ipip_conscientiousness_category','label_ipip_stability_category', 'label_ipip_intellect_category', 'label_stai_stress_category', 'label_panas_negative_affect', 'label_ttm_stage', 'label_breq_self_determination'])

In [64]:
train_data, test_data = preprocessing_functions.train_test_split_per_user(extraversion)
fold_groups = train_data.id
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

In [65]:
s = setup(data=train_data, target='label_ipip_extraversion_category', silent=True, session_id=123,
          fold_strategy='groupkfold', fold=3,  fold_groups=fold_groups, test_data=test_data)

,Description,Value
0,session_id,123
1,Target,label_ipip_extraversion_category
2,Target Type,Multiclass
3,Label Encoded,"AVERAGE: 0, HIGH: 1, LOW: 2"
4,Original Data,"(5199, 43)"
5,Missing Values,True
6,Numeric Features,33
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


In [66]:
best = compare_models(sort = '%s'%metric) # change to F1 and AUC

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.4007,0.5762,0.4273,0.5204,0.3890,0.1159,0.1316,0.0833
lr,Logistic Regression,0.2511,0.5254,0.3740,0.1610,0.1697,0.0435,0.0600,0.1233
dummy,Dummy Classifier,0.2371,0.5000,0.3333,0.0781,0.1128,0.0000,0.0000,0.1100
knn,K Neighbors Classifier,0.2818,0.4843,0.3134,0.3661,0.2918,-0.0383,-0.0405,0.1567
lightgbm,Light Gradient Boosting Machine,0.1998,0.4202,0.1658,0.2294,0.2048,-0.2074,-0.2262,1.6533
dt,Decision Tree Classifier,0.2321,0.4127,0.2341,0.2702,0.2348,-0.1468,-0.1601,0.0933
gbc,Gradient Boosting Classifier,0.2343,0.3814,0.1894,0.2445,0.2350,-0.1519,-0.1734,2.1567
rf,Random Forest Classifier,0.2601,0.3696,0.2535,0.2699,0.2452,-0.1100,-0.1270,0.3100
qda,Quadratic Discriminant Analysis,0.1705,0.3686,0.1551,0.2088,0.1776,-0.2236,-0.2715,0.1933
lda,Linear Discriminant Analysis,0.2074,0.3659,0.1776,0.2408,0.2104,-0.2016,-0.2218,0.0967


In [67]:
extraversion = df_pre.drop(columns=['date', 'label_sema_negative_feelings', 'label_ipip_agreeableness_category', 'label_ipip_conscientiousness_category','label_ipip_stability_category', 'label_ipip_intellect_category', 'label_stai_stress_category', 'label_panas_negative_affect', 'label_ttm_stage', 'label_breq_self_determination'])

In [68]:
train_data, test_data = preprocessing_functions.train_test_split_per_user(extraversion)
fold_groups = train_data.id
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

In [69]:
s = setup(data=train_data, target='label_ipip_extraversion_category', silent=True, session_id=123,
          fold_strategy='groupkfold', fold=3,  fold_groups=fold_groups, test_data=test_data,
          normalize=True, normalize_method='minmax', fix_imbalance=True, remove_multicollinearity = True, multicollinearity_threshold = 0.6)

,Description,Value
0,session_id,123
1,Target,label_ipip_extraversion_category
2,Target Type,Multiclass
3,Label Encoded,"AVERAGE: 0, HIGH: 1, LOW: 2"
4,Original Data,"(5199, 148)"
5,Missing Values,True
6,Numeric Features,107
7,Categorical Features,40
8,Ordinal Features,False
9,High Cardinality Features,False


In [70]:
best = compare_models(sort = '%s'%metric) # change to F1 and AUC

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.3160,0.5517,0.3286,0.3938,0.3213,-0.0124,-0.0166,1.4500
lda,Linear Discriminant Analysis,0.3509,0.5474,0.3510,0.3720,0.3404,0.0266,0.0297,0.2567
ada,Ada Boost Classifier,0.4128,0.5386,0.4263,0.3913,0.3702,0.0900,0.1064,1.4433
qda,Quadratic Discriminant Analysis,0.3221,0.5176,0.4090,0.4938,0.3356,0.0328,0.0490,0.7100
nb,Naive Bayes,0.3128,0.5169,0.2935,0.4140,0.3360,-0.0456,-0.0487,0.4333
rf,Random Forest Classifier,0.2039,0.5065,0.2362,0.3712,0.2128,-0.1177,-0.1343,0.8833
dummy,Dummy Classifier,0.3034,0.5000,0.3333,0.0943,0.1433,0.0000,0.0000,0.1733
gbc,Gradient Boosting Classifier,0.3010,0.4751,0.3381,0.3980,0.3086,0.0444,0.0517,11.8433
et,Extra Trees Classifier,0.2445,0.4640,0.2658,0.4816,0.2725,-0.0785,-0.0851,0.3467
dt,Decision Tree Classifier,0.2182,0.4430,0.2479,0.3964,0.2176,-0.1221,-0.1218,0.4233


In [71]:
df = pd.read_pickle('../data/unified_dataframe/data_preprocessed.pkl')
extraversion = df.drop(columns=['date', 'label_sema_negative_feelings', 'label_ipip_agreeableness_category', 'label_ipip_conscientiousness_category','label_ipip_stability_category', 'label_ipip_intellect_category', 'label_stai_stress_category', 'label_panas_negative_affect', 'label_ttm_stage', 'label_breq_self_determination'])
extraversion = extraversion[extraversion['label_ipip_extraversion_category'].notna()]
y = extraversion['label_ipip_extraversion_category']

In [72]:
cols = ['stress_score', 'steps', 'sleep_duration']
for col in cols:
    extracted_features = extract_features(df, column_id="id", column_sort="date",
                                          n_jobs=5, column_kind=None, column_value=col,
                                          show_warnings=False)
    extracted_features = extracted_features.reset_index()
    extracted_features = extracted_features.rename(columns={"index": "id"})
    extraversion = extraversion.merge(extracted_features, how='left', on=['id'])

Feature Extraction: 100%|██████████| 24/24 [00:16<00:00,  1.43it/s]


In [73]:
train_data, test_data = preprocessing_functions.train_test_split_per_user(extraversion)
fold_groups = train_data.id
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

In [74]:
s = setup(data=train_data, target='label_ipip_extraversion_category', silent=True, session_id=123,
          fold_strategy='groupkfold', fold=3,  fold_groups=fold_groups, test_data=test_data,
          normalize=True, normalize_method='minmax', fix_imbalance=True, remove_multicollinearity = True, multicollinearity_threshold = 0.6)

,Description,Value
0,session_id,123
1,Target,label_ipip_extraversion_category
2,Target Type,Multiclass
3,Label Encoded,"AVERAGE: 0, HIGH: 1, LOW: 2"
4,Original Data,"(3985, 2515)"
5,Missing Values,True
6,Numeric Features,2240
7,Categorical Features,274
8,Ordinal Features,False
9,High Cardinality Features,False


In [75]:
best = compare_models(sort = '%s'%metric) # change to F1 and AUC

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.3050,0.6172,0.3725,0.2613,0.2543,0.0968,0.1032,3.9700
lightgbm,Light Gradient Boosting Machine,0.1586,0.5838,0.1926,0.3539,0.1491,-0.1051,-0.1315,1.9767
lr,Logistic Regression,0.3102,0.5785,0.4433,0.2825,0.2617,0.1224,0.1476,6.3500
et,Extra Trees Classifier,0.2025,0.5447,0.3435,0.1259,0.1339,0.0115,0.0246,0.7667
dummy,Dummy Classifier,0.3092,0.5000,0.3333,0.1339,0.1786,0.0000,0.0000,0.6767
qda,Quadratic Discriminant Analysis,0.3092,0.4977,0.3018,0.2134,0.2159,0.0030,-0.0529,8.9267
dt,Decision Tree Classifier,0.2027,0.4976,0.2172,0.5487,0.2323,-0.0387,-0.0593,1.5333
gbc,Gradient Boosting Classifier,0.2411,0.4840,0.2215,0.3320,0.2550,-0.0451,-0.0588,9.0000
knn,K Neighbors Classifier,0.2317,0.4782,0.3327,0.3407,0.1837,-0.0310,-0.0284,8.3233
lda,Linear Discriminant Analysis,0.2048,0.4459,0.2677,0.3214,0.2084,-0.0646,-0.0901,0.7433


In [76]:
d = pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(best.coef_[0])}).sort_values(by='Value', ascending=False)

AttributeError: 'GaussianNB' object has no attribute 'coef_'

In [ ]:
pd.set_option('display.max_rows', 500)
d.reset_index(inplace=True)
d

In [ ]:
d['Feature']